In [18]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
mnist = input_data.read_data_sets('./MNIST_data',one_hot=True)

Extracting ./MNIST_data\train-images-idx3-ubyte.gz
Extracting ./MNIST_data\train-labels-idx1-ubyte.gz
Extracting ./MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data\t10k-labels-idx1-ubyte.gz


In [19]:
mnist.train.num_examples

55000

In [20]:
trX,trY,teX,teY = mnist.train.images,mnist.train.labels,mnist.test.images,mnist.test.labels

In [21]:
print(trX.shape)

(55000, 784)


In [22]:
print(trY.shape)
trY

(55000, 10)


array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.]])

In [23]:
print(teX.shape)
teX

(10000, 784)


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [24]:
print(teY.shape)
teY

(10000, 10)


array([[ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

#### 1.处理输入的数据

##### 需要把上面的trX和trY的形状变成[-1,28,28,1],-1表示不考虑输入图片的数量，28*28 表示图片的长和宽
##### 1是颜色通道。MNIST是黑白的，颜色通道是1，RGB是3

In [25]:
trX = trX.reshape(-1,28,28,1)
teX = teX.reshape(-1,28,28,1)

In [26]:
X = tf.placeholder(tf.float32,[None,28,28,1])
Y = tf.placeholder(tf.float32,[None,10])

In [27]:
#2.定义初始化函数
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape,stddev=0.1))

In [28]:
#3.初始化权重,设置卷积核的大小为3*3
w = init_weights([3,3,1,32])
w2 = init_weights([3,3,32,64])
w3 = init_weights([3,3,64,128])
w4 = init_weights([128*4*4,625])
w_o = init_weights([625,10])

In [29]:
#3.神经网络的构建
# X是输入数据，w每一层的权重，keep_conv，keep_hidden保留神经元比例
def model(X,w,w2,w3,w4,w_o,keep_conv,keep_hidden):
    #第一组卷积层及池化层，最后dropout一些神经元
    conv_relu_1 = tf.nn.relu(tf.nn.conv2d(X,w,strides=[1,1,1,1],padding='SAME'))
    max_pool_1 = tf.nn.max_pool(conv_relu_1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    drop_out_1 = tf.nn.dropout(max_pool_1,keep_conv)
    
    #第二组卷积层及池化层，最后dropout一些神经元
    conv_relu_2 = tf.nn.relu(tf.nn.conv2d(drop_out_1,w2,strides=[1,1,1,1],padding='SAME'))
    max_pool_2 = tf.nn.max_pool(conv_relu_2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    drop_out_2 = tf.nn.dropout(max_pool_2,keep_conv)
    
    #第三层卷积层及池化层，最后dropout一些神经元
    conv_relu_3 = tf.nn.relu(tf.nn.conv2d(drop_out_2,w3,strides=[1,1,1,1],padding='SAME'))
    max_pool_3 = tf.nn.max_pool(conv_relu_3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    #reshape
    max_pool_3 = tf.reshape(max_pool_3,[-1,w4.get_shape().as_list()[0]])
    drop_out_3 = tf.nn.dropout(max_pool_3,keep_conv)
    
    #全连接层，最后dropout一些神经元
    l4 = tf.nn.relu(tf.matmul(drop_out_3,w4))
    l4 = tf.nn.dropout(l4,keep_hidden)
    
    #输出层
    out = tf.matmul(l4,w_o)
    
    #返回预测值
    return out

In [30]:
#4.定义占位符
p_keep_conv = tf.placeholder(tf.float32)
p_keep_hidden = tf.placeholder(tf.float32)

predict_result = model(X,w,w2,w3,w4,w_o,p_keep_conv,p_keep_hidden)

In [31]:
#5.定义损失函数和优化器
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predict_result,labels=Y))
#学习率为0.00001,衰减值为0.9
train = tf.train.RMSPropOptimizer(0.0001,0.9).minimize(cost)
predict = tf.argmax(predict_result,1)

In [32]:
#6.训练模型
batch_size = 10
test_size = 10


In [33]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(10):
        training_batch = zip(range(0,len(trX),batch_size),range(batch_size,len(trX)+1,batch_size))
        for start,end in training_batch:
            sess.run(train,feed_dict={X:trX[start:end],Y:trY[start:end],p_keep_conv:0.5,p_keep_hidden:0.5})
    test_indices = np.arange(len(trX))
    np.random.shuffle(test_indices)
    test_indices = test_indices[0:test_size]
    print(i,np.mean(np.argmax(test_indices),axis=1) == sess.run(predict,feed_dict={X:teX[test_indices],p_keep_conv:1.0,p_keep_hidden:1.0}))

IndexError: tuple index out of range